# RT Prediction
### Targeted Analysis
See the [Targeted_Analysis.md](https://github.com/biorack/metatlas/blob/main/docs/Targeted_Analysis.md) file on GitHub for documentation on how to use this notebook.

#### Parameters
The next code block sets parameters that are used throughout the remainder of the notebook.

In [ ]:
# pylint: disable=invalid-name,missing-module-docstring

# one of 'positive' or 'negative'
# uses this polarity for RT prediction, but will generate subsequent notebooks for both polarities
polarity = "positive"

# an integer, increment if you need to redo your analysis
# will be appended to your username to create analysis_id
analysis_number = 0

# experiment ID that must match the parent folder containing the LCMS output files
# An example experiment ID is '20201116_JGI-AK_LH_506489_SoilWarm_final_QE-HF_HILICZ_USHXG01530'
experiment = "REPLACE ME"

# group will only be used in RT prediction if their name has a substring match to this list of strings
include_groups = None

# Exclude files with names containing any of the substrings in this list. Eg., ['peas', 'beans']
exclude_files = None

# Exclude groups with names containing any of the substrings in this list.
# 'POS' or 'NEG' will be auto-appended later, so you shouldn't use them here.
exclude_groups = None

# list of substrings that will group together when creating groups
# this provides additional grouping beyond the default grouping on field #12
groups_controlled_vocab = ["QC", "InjBl", "ISTD"]

# thresholds for filtering out compounds with weak MS1 signals
# set to None to disable a filter
num_points = 5
peak_height = 4e5

# threshold for filtering out compounds with poor MS2 spectra similaritiy
# Should be a value in range 0 to 1. Set to None to disable this filter.
msms_score = None

# Override the rt_min and rt_max values in the atlas
# both rt_min_delta and rt_max_delta are *added* to rt_peak, so rt_min_delta <= rt_max_delta.
# Normally you will have rt_min_delta < 0 and rt_max_delta > 0
# but you can have both of them be positive or both negative for extreme cases.
# Set to None to start with the rt_min and rt_max values from the atlas
# Only impacts the atlas used for RT prediction, not subsequent atlases.
rt_min_delta = None
rt_max_delta = None

# if True, use a 2nd order polynomial model for RT prediction.
# if False, use a liner model
use_poly_model = True

# if True, don't generate any atlases or notebooks
# if False, an atlas with adjusted RTs and follow up notebooks will be generated
# this parameter is deprecated, transition to using stop_before instead
model_only = False

# One of 'qc_plots', 'atlases', 'notebooks', 'msms_hits', None
# Terminates processing of RT_Prediction.ipynb early
# model_only overrides this setting
# model_only = True is equivalent to stop_before = 'qc_plots'
# None generates all outputs
# normal processing order is:
# 1. generate model
#    makes Actual_vs_Predicted_RTs.pdf, RT_Predicted_Model_Comparison.csv, rt_model.txt
# 2. create QC_plots
#.   makes all other plots in data_QC such as EICs, mirror plots, Compound_Atlas_*.pdf
# 3. create RT adjusted atlases
# 4. create notebooks for subsequent analysis
# 5. generate MSMS hits for subsequent notebooks and filter atlases
stop_before = None

# The QC run or name of the summary statistic generated from all QC runs
# that will be used as the data source for the dependent variable for RT model generation.
# Can be the name of a summary statistic generated from all QC runs:
# "median", "mean", "min", max"
# Or specific QC run, by supply name of an h5 file (without the path)
dependent_data_source = "median"

# List of InChi Keys to be ignored when creating the RT prediction model.
inchi_keys_not_in_model = None


# The rest of this block contains project independent parameters

# to use an older version of the metatlas source code, set this to a commit id,
# branch name, or tag. If None, then use the the "main" branch.
source_code_version_id = None

# Full path to the directory where you want this notebook to store data.
# A subdirectory will be auto created within this directory for each project.
# You can place this anywhere on cori's filesystem, but placing it within your
# global home directory is recommended so that you do not need to worry about
# your data being purged. Each project will take on the order of 100 MB.
project_directory = "/global/homes/FIRST-INITIAL-OF-USERNAME/USERNAME/metabolomics_projects"

# ID from Google Drive URL for base output folder .
# The default value is the ID that corresponds to 'JGI_Metabolomics_Projects'.
google_folder = "0B-ZDcHbPi-aqZzE5V3hOZFc0dms"

# maximum number of CPUs to use
# when running on jupyter.nersc.gov, you are not allowed to set this above 4
max_cpus = 4

# Threshold for how much status information metatlas functions print in the notebook
# levels are 'DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL'
log_level = "INFO"

In [ ]:
# pylint: disable=wrong-import-position,import-error,missing-class-docstring
import logging  # noqa: E402
from pathlib import Path  # noqa: E402


class StopExecution(Exception):
    def _render_traceback_(self):
        pass


logger = logging.getLogger("metatlas.jupyter")
kernel_def = """{"argv":["shifter","--entrypoint","--image=wjhjgi/metatlas_shifter:latest","/usr/local/bin/python","-m",
                 "ipykernel_launcher","-f","{connection_file}"],"display_name": "Metatlas Targeted","language": "python",
                 "metadata": { "debugger": true }}"""
kernel_file_name = Path.home() / ".local" / "share" / "jupyter" / "kernels" / "metatlas-targeted" / "kernel.json"
try:
    has_root_kernel = Path("/root/.local/share/jupyter/kernels/papermill/kernel.json").is_file()
except PermissionError:
    has_root_kernel = False
if not has_root_kernel and not kernel_file_name.is_file():
    kernel_file_name.parent.mkdir(parents=True, exist_ok=True)
    with kernel_file_name.open(mode="w", encoding="utf-8") as f:
        f.writelines(kernel_def)
    logger.critical('CRITICAL: Notebook kernel has been installed. Set kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution
try:
    from metatlas.tools import notebook  # noqa: E402
except ImportError as err:
    logger.critical('CRITICAL: Set notebook kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution from err
notebook.setup(log_level, source_code_version_id)
from metatlas.tools import notebook, predict_rt  # noqa: E402

In [ ]:
ids = predict_rt.get_analysis_ids_for_rt_prediction(
    experiment,
    project_directory,
    google_folder,
    analysis_number,
    polarity,
    exclude_files,
    include_groups,
    exclude_groups,
    groups_controlled_vocab,
)

In [ ]:
predict_rt.generate_outputs(
    ids,
    max_cpus,
    num_points=num_points,
    peak_height=peak_height,
    msms_score=msms_score,
    use_poly_model=use_poly_model,
    model_only=model_only,
    selected_col=dependent_data_source,
    stop_before=stop_before,
    source_code_version_id=source_code_version_id,
    rt_min_delta=rt_min_delta,
    rt_max_delta=rt_max_delta,
    inchi_keys_not_in_model=inchi_keys_not_in_model,
)